In [16]:
import pandas as pd
import datetime
from Preprocessing import Preprocessing

#file_path = 'data/fixations.csv'
#df = pd.read_csv(file_path, delimiter=";")
filename = 'fixations.csv'
delimiter = ','
df = Preprocessing.load_data_csv(filename, delimiter)

-- loading dataset ...
-- found file 
-- done


In [24]:
# 1906 records in total, access the elements: df['start timestamp [ns]'][0] ~ df['start timestamp [ns]'][1906]
# df['end timestamp [ns]'][0] - df['start timestamp [ns]'][0]
timestamp = df['start timestamp [ns]'][0]
timestamp_seconds = timestamp // 10**9 # Convert nanoseconds to seconds 
timestamp_nanoseconds = timestamp % 10**9 # Extract remaining nanoseconds 
# Convert to datetime object 
dt = datetime.datetime.utcfromtimestamp(timestamp_seconds) # Print the datetime object in a desired format 
print(dt.strftime("%Y-%m-%d %H:%M:%S") + f".{timestamp_nanoseconds}")

2023-05-12 08:19:44.205738074
